In [57]:
import pandas as pd
import numpy as np
import os

from googletrans import Translator

In [31]:
# Path to the folder containing the CSV files
folder_path = './data'

# List to hold individual dataframes
dataframes = []

# Iterate over all files in the folder
for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):
        # Construct full file path
        file_path = os.path.join(folder_path, filename)
        
        # Read the CSV file into a DataFrame
        df = pd.read_csv(file_path)
        
        # Extract the place name from the filename (removing the '.csv' extension)
        place_name = filename.replace('.csv', '').replace('_', ' ')
        
        # Add the place column to the DataFrame
        df['Place name'] = place_name
        
        # Append the DataFrame to the list
        dataframes.append(df)

# Combine all DataFrames into one
data_review = pd.concat(dataframes, ignore_index=True)

# Save the combined DataFrame to a new CSV file
data_review.to_csv('combined_scrap_result.csv', index=False)

# Display the first few rows of the combined DataFrame
display(data_review.head())

,nama,bintang,komentar,Waktu,Place name
0,Satria Sihombing,5.0,view nya menyala 🤩🔥🔥,an hour,Bobocabin Baturraden Purwokerto
1,silas nainggolan,5.0,Penginapan yang paling menyatu dengan alam. Ca...,an hour,Bobocabin Baturraden Purwokerto
2,Ari Setiawan,5.0,"Bagus, hotel bernuasna baru yang berteknologi ...",2 hours,Bobocabin Baturraden Purwokerto
3,mutiara saragih,5.0,good,2 hours,Bobocabin Baturraden Purwokerto
4,hendro sebayang,5.0,So an exciting stay experience,2 hours,Bobocabin Baturraden Purwokerto


In [32]:
print(data_review.size)
data_review.dropna(subset="komentar", inplace=True)
print(data_review.size)

58195
33885


In [36]:
data_review.reset_index(drop=True, inplace=True)
data_review

,nama,bintang,komentar,Waktu,Place name
0,Satria Sihombing,5.0,view nya menyala 🤩🔥🔥,an hour,Bobocabin Baturraden Purwokerto
1,silas nainggolan,5.0,Penginapan yang paling menyatu dengan alam. Ca...,an hour,Bobocabin Baturraden Purwokerto
2,Ari Setiawan,5.0,"Bagus, hotel bernuasna baru yang berteknologi ...",2 hours,Bobocabin Baturraden Purwokerto
3,mutiara saragih,5.0,good,2 hours,Bobocabin Baturraden Purwokerto
4,hendro sebayang,5.0,So an exciting stay experience,2 hours,Bobocabin Baturraden Purwokerto
...,...,...,...,...,...
6772,pohon bumi,4.0,"All digital technologies, capsule/pod hotel 🏨,...",2 months,Bobopod Thamrin Jakarta
6773,Trip.com Member,2.7,"Access only by stair, can not keep lugage, if ...",3 months,Bobopod Thamrin Jakarta
6774,Sandi Prabowo,5.0,2 hari nginep disini ngga mengecewakan. Keluar...,3 months,Bobopod Thamrin Jakarta
6775,suka review,5.0,Good places to stay deket St. Sudirman didepan...,4 months,Bobopod Thamrin Jakarta


In [25]:
slang_df = pd.read_csv('colloquial-indonesian-lexicon.csv')
slang_df = slang_df.iloc[:, :2]

slang_dict = pd.Series(slang_df.formal.values, index=slang_df.slang).to_dict()
slang_dict

{'woww': 'wow',
 'aminn': 'amin',
 'met': 'selamat',
 'netaas': 'menetas',
 'keberpa': 'keberapa',
 'eeeehhhh': 'eh',
 'kata2nyaaa': 'kata-katanya',
 'hallo': 'halo',
 'kaka': 'kakak',
 'ka': 'kak',
 'daah': 'dah',
 'aaaaahhhh': 'ah',
 'yaa': 'ya',
 'smga': 'semoga',
 'slalu': 'selalu',
 'amiin': 'amin',
 'kk': 'kakak',
 'trus': 'terus',
 'sii': 'sih',
 'nyenengin': 'menyenangkan',
 'bgt': 'banget',
 'gemess': 'gemas',
 'akuuu': 'aku',
 'jgn': 'jangan',
 'udah': 'sudah',
 'gitu': 'begitu',
 'aja': 'saja',
 'gemesiin': 'menggemaskan',
 'menyenangkn': 'menyenangkan',
 'rb': 'ribu',
 'akau': 'aku',
 'saranin': 'menyarankan',
 'nemuin': 'menemukan',
 'yah': 'ya',
 'mnis2nya': 'manis-manisnya',
 'gtu': 'begitu',
 'tu': 'itu',
 'idolaa': 'idola',
 'bangett': 'banget',
 'kaakk': 'kak',
 'baaiikkk': 'baik',
 'telaatt': 'telat',
 'kaak': 'kak',
 'yg': 'yang',
 'lg': 'lagi',
 'skarang': 'sekarang',
 'sempet': 'sempat',
 'karna': 'karena',
 'ngga': 'enggak',
 'mantep': 'mantap',
 'loohhh': 'loh',

In [26]:
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import warnings, logging, regex as re

warnings.filterwarnings("ignore")
logging.basicConfig(level=logging.INFO)

slang_df = pd.read_csv('colloquial-indonesian-lexicon.csv')
slang_df = slang_df.iloc[:, :2]

slang_dict = pd.Series(slang_df.formal.values, index=slang_df.slang).to_dict()
slang_dict

def clean_text(data: pd.DataFrame) -> pd.DataFrame:
    
    def remove_trailing_char(st):
        s_arr = st.split(" ")
        hasil = []
        for s in s_arr:
            unique_char = set(s)
            for char in unique_char:
                if s.count(char) > 2:
                    s = s.replace(char*s.count(char), char)
            hasil.append(s)

        return " ".join(hasil)

    def normalize_text(text):
        # Create regex pattern to match the slang words
        pattern = re.compile(r'\b(' + '|'.join(re.escape(key) for key in slang_dict.keys()) + r')\b')
        
        # Replace slang words with formal words using the dictionary
        normalized_text = pattern.sub(lambda x: slang_dict[x.group()], text)
        
        return normalized_text

    stemmer = StemmerFactory().create_stemmer()
    stopword = StopWordRemoverFactory().create_stop_word_remover()

    logging.info("removing null..")
    data = data.dropna()
    logging.info("lowercasing..")
    data["komentar"] = data["komentar"].str.lower()
    logging.info("removing google translate..")
    data["komentar"] = data["komentar"].apply(
        lambda x: x.replace("(diterjemahkan oleh google)", ""))
    logging.info("removing newline..")
    data["komentar"] = data["komentar"].apply(lambda x: x.replace("\n", " "))
    logging.info("removing punctuation..")
    data["komentar"] = data["komentar"].str.replace('[^\w\s]', ' ')
    data["komentar"] = data["komentar"].str.replace('²', ' ')
    logging.info("Normalize Slang..")
    data["komentar"] = data["komentar"].apply(
        lambda x: normalize_text(x))
    logging.info("stripping..")
    data["komentar"] = data["komentar"].apply(lambda x: x.strip())
    logging.info("stemming..")
    data["komentar"] = data["komentar"].apply(stemmer.stem)
    logging.info("remove stopword..")
    data["komentar"] = data["komentar"].apply(stopword.remove)
    logging.info("removing traling char..")
    data["komentar"] = data["komentar"].apply(
        lambda x: remove_trailing_char(x))

    return data

In [ ]:
import time 

start_time = time.process_time() 

data_review_clean = clean_text(data_review)
data_review["clean_text"] = data_review_clean["komentar"]

end_time = time.process_time() 

print(f"Waktu CPU yang digunakan: {end_time - start_time} detik")

data_review.to_csv('cleaned_data.csv', index=False)

data_review

In [3]:
from deep_translator import GoogleTranslator

combined_df = pd.read_csv('cleaned_data.csv')

def translate_to_english(text):
    try:
        if text and isinstance(text, str):
            translated = GoogleTranslator(source='auto', target='en').translate(text)
            return translated
        return text
    except Exception as e:

        print(f"Error translating text: {e}, The text : {text}")
        return text

combined_df['komentar'] = combined_df['clean_text'].apply(translate_to_english)

combined_df.to_csv('combined_English_scrap_result.csv', index=False)

print(combined_df.head())

Error translating text: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')), The text : 2 hari nginep sini enggak kecewa keluar hotel banyak jajan dekat bagai fasilitas tunjang pergi
               nama  bintang  \
0  Satria Sihombing      5.0   
1  silas nainggolan      5.0   
2      Ari Setiawan      5.0   
3   mutiara saragih      5.0   
4   hendro sebayang      5.0   

                                            komentar    Waktu  \
0                                     the view is on  an hour   
1  stay at least one of the natural cabins, the s...  an hour   
2  nice hotel with a new, high technology feel, v...  2 hours   
3                                               good  2 hours   
4                    sok an exciting stay experience  2 hours   

                        Place name  \
0  Bobocabin Baturraden Purwokerto   
1  Bobocabin Baturraden Purwokerto   
2  Bobocabin Baturraden Purwokerto   
3  Bobocabin Baturraden Purwokerto   
4 

In [65]:
from sklearn.feature_extraction.text import CountVectorizer
tes = pd.read_csv('cleaned_data.csv')

tes.dropna(subset="clean_text", inplace=True)

# Display the rows with null values
# print(rows_with_nulls)
# Create a count vectorizer
vectorizer = CountVectorizer(max_df=0.95, min_df=2, stop_words='english')

# Fit and transform the cleaned reviews
dtm = vectorizer.fit_transform(tes['clean_text'])
dtm

<6732x5689 sparse matrix of type '<class 'numpy.int64'>'
	with 142636 stored elements in Compressed Sparse Row format>

In [66]:
from sklearn.decomposition import LatentDirichletAllocation

# Set the number of topics
num_topics = 5

# Create LDA model
lda = LatentDirichletAllocation(n_components=num_topics, random_state=42)

# Fit the LDA model
lda.fit(dtm)

# Display the topics
for index, topic in enumerate(lda.components_):
    print(f'Topic #{index + 1}:')
    print([vectorizer.get_feature_names_out()[i] for i in topic.argsort()[-20:]])

Topic #1:
['cukup', 'buat', 'lokasi', 'sini', 'alam', 'enggak', 'jalan', 'malam', 'view', 'tempat', 'inap', 'jadi', 'makan', 'banget', 'kamar', 'sangat', 'nya', 'bagus', 'bobocabin', 'cabin']
Topic #2:
['kamar', 'harga', 'bagus', 'cocok', 'nya', 'inap', 'hotel', 'buat', 'kota', 'layan', 'bobobox', 'ramah', 'strategis', 'dekat', 'lokasi', 'banget', 'sangat', 'bersih', 'tempat', 'nyaman']
Topic #3:
['yang', 'kalau', 'sudah', 'jauh', 'cuma', 'bobobox', 'ada', 'barang', 'kamar', 'bayar', 'orang', 'bawa', 'jam', 'check', 'hotel', 'mau', 'mobil', 'jadi', 'enggak', 'parkir']
Topic #4:
['bobobox', 'just', 'really', 'orang', 'service', 'location', 'view', 'hotel', 'staff', 'great', 'experience', 'clean', 'nice', 'cabin', 'stay', 'sok', 'si', 'room', 'place', 'good']
Topic #5:
['ada', 'pakai', 'air', 'toilet', 'nyaman', 'bobobox', 'sih', 'makan', 'tempat', 'sama', 'di', 'kalo', 'buat', 'jadi', 'banget', 'bersih', 'nya', 'mandi', 'enggak', 'kamar']


In [ ]:
from deep_translator import GoogleTranslator

# Load the combined data
combined_df = pd.read_csv('cleaned_data.csv')

# Convert specified columns to string
# combined_df['nama'] = combined_df['nama'].astype(str)
# combined_df['komentar'] = combined_df['komentar'].astype(str)
# combined_df['Waktu'] = combined_df['Waktu'].astype(str)

# Function to translate text to Indonesian
def translate_to_indonesian(text):
    try:
        if text and isinstance(text, str):
            translated = GoogleTranslator(source='auto', target='id').translate(text)
            return translated
        return text
    except Exception as e:

        print(f"Error translating text: {e}, The text : {text}")
        return text

# Apply the translation function to the 'komentar' column
combined_df['komentar'] = combined_df['komentar'].apply(translate_to_indonesian)

# Save the translated DataFrame to a new CSV file
combined_df.to_csv('combined_Indonesia_scrap_result.csv', index=False)

# Display the first few rows of the translated DataFrame
print(combined_df.head())

Error translating text: ("Connection broken: ConnectionAbortedError(10053, 'An established connection was aborted by the software in your host machine', None, 10053, None)", ConnectionAbortedError(10053, 'An established connection was aborted by the software in your host machine', None, 10053, None)), The text : dingiiiiinn bangeeett.. mantaaaaabb.. hujan terus tapi hari ini.. sepanjang hari..
Error translating text: HTTPSConnectionPool(host='translate.google.com', port=443): Max retries exceeded with url: /m?tl=id&sl=auto&q=Apotik+tutup (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x0000021AEF7C5F10>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed')), The text : Apotik tutup
Error translating text: HTTPSConnectionPool(host='translate.google.com', port=443): Max retries exceeded with url: /m?tl=id&sl=auto&q=Pemandangan+nya+bagus+%2C+kamar+nya+bersih+%2C+makanan+nya+enak+%2C+staff+nya+ramah+sigap+terimakasih+pelayanan+nya+%2C+cocok